In [1]:
import gc
import pandas as pd
import numpy as np
import os
import xgboost as xgb
import lightgbm as lgb
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.metrics import roc_curve 
def score(y,pred): 
    fpr, tpr, thresholds = roc_curve(y, pred, pos_label=1) 
    score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    return score 
# from datetime import date
# def getWeekday(row):
#     if row == 'null':
#         return row
#     else:
#         return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

In [2]:
####################训练集#######################
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [19]:
#data.sort_values("date",inplace=True)

In [3]:
####################训练集#######################
pd.set_option('display.max_columns',2000)
data

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,0,20171103,0,0,0,0,100807.0,0,5,1,1,1,1,2,0,2,0,2,2,2,1,31.0,61.0,142.0,245.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,96.0,38.0,301.0,302.0,302.0,302.0,301.0,302.0,40.0,79.0,40.0,79.0,2.0,3.0,4.0,3.0,41.0,80.0,96.5,721.1,3252.0,154.5,394.5,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,8.0,15.0,30.0,4.0,14.0,34.0,70.0,142.0,3.0,15.0,41.0,79.0,191.0,192.0,190.0,190.0,190.0,190.0,191.0,192.0,191.0,190.0,190.0,190.0,32.0,33.0,25.0,25.0,24.0,25.0,31.0,79.0,42.0,42.0,42.0,68.0,32.0,32.0,28.0,27.0,28.0,27.0,32.0,235.0,235.0,126.0,126.0,127.0,126.0,235.0,192.0,191.0,190.0,189.0,189.0,301.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,14.0,41.0,80.0,167.0,2.0,2.0,4.0,3.0,3.0,4.0,6.0,6.0,2.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,4.0,1.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,10.0,16.0,2.0,2.0,4.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,27.0,297.0,302.0,384.0,770.0,20.0,99.0,102.0,124.0,140.0,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,0,20170917,0,1,1,1,100805.0,1,5,2,1,1,0,2,1,2,2,1,1,2,0,14.0,13.0,48.0,48.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,4.0,4.0,5.0,1.0,1.0,4.0,3.0,3.0,79.0,12.0,302.0,32.0,301.0,302.0,302.0,302.0,23.0,26.0,23.0,26.0,2.0,2.0,3.0,4.0,23.0,26.0,5.0,717.4,733.3,231.8,472.8,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,4.0,8.0,11.0,12.0,4.0,12.0,23.0,26.0,27.0,5.0,12.0,24.0,27.0,122.0,121.0,16.0,121.0,121.0,122.0,121.0,122.0,121.0,16.0,121.0,122.0,32.0,31.0,13.0,27.0,26.0,27.0,32.0,79.0,40.0,39.0,39.0,63.0,31.0,31.0,15.0,28.0,29.0,29.0,31.0,231.0,232.0,17.0,99.0,99.0,98.0,232.0,121.0,122.0,17.0,121.0,121.0,302.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,5.0,12.0,24.0,26.0,27.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1

In [4]:
data.loc[data['label'] == -1, 'label'] = 1

In [5]:
data.loc[data['label'] == 1, 'label'] = 2

In [6]:
data['label'].value_counts()

0    977884
2     16847
Name: label, dtype: int64

In [7]:
data.loc[data['label'] == 0, 'label'] = 1

In [8]:
data['label'].value_counts()

1    977884
2     16847
Name: label, dtype: int64

In [9]:
data.loc[data['label'] == 2, 'label'] = 0
data['label'].value_counts()

1    977884
0     16847
Name: label, dtype: int64

In [10]:
#data['day'] = data['date'].apply(lambda x: int(str(x)[6:8]))

In [10]:
####################测试集#######################
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_b.csv")
data_test = pd.read_csv(csv_path)

In [11]:
####################测试集#######################
data_test

,id,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,661ae9a03c6c74feab2555c9987e140ae3f5421bd8e7a0...,20180305,0,0,1,1,100810.0,0,4,2,0,1,0,2,1,1,1,0,2,2,0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,4.0,3.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,5.0,5.0,10.0,24.0,35.0,46.0,1.0,2.0,2.0,4.0,3.0,4.0,11.0,11.0,2.0,1.0,2.0,2.0,2.0,1.0,3.0,4.0,2.0,2.0,3.0,2.0,1.0,3.0,4.0,2.0,2.0,8.0,7.0,7.0,2.0,1.0,2.0,2.0,3.0,2.0,3.0,7.0,10.0,15.0,2.0,1.0,3.0,4.0,9.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,78.0,302.0,302.0,324.0,353.0,44.0,150.0,149.0,156.0,155.0,301.0,301.0,302.0,192.0,301.0,218.0,219.0,219.0,153.0,218.0
1,ada98d4358e72a27cc5e92f691a87a4fd62a7cda0387e2...,20180303,0,0,1,1,100810.0,1,6,1,0,2,1,1,0,2,2,0,2,2,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,27.0,4.0,6.0,6.0,9.0,24.0,51.0,78.0,51.0,78.0,4.0,6.0,7.0,9.0,55.0,82.0,686.5,14090.8,20128.8,1392.5,4811.3,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,4.0,8.0,15.0,22.0,8.0,23.0,56.0,83.0,109.0,9.0,23.0,52.0,80.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,25.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,6.0,9.0,23.0,53.0,80.0,107.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,7.0,23.0,25.0,5.0,6.0,2.0,1.0,1.0,2.0,2.0
2,e33675a962e5bf44d05a2b01903a4beb88a0c6385c05c6...,20180226,0,0,1,1,100811.0,1,2,1,0,1,0,

In [12]:
# data['weekday'] = data['date'].astype(str).apply(getWeekday)
# data_test['weekday'] = data_test['date'].astype(str).apply(getWeekday)
# # change weekday to one-hot encoding 
# weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
# print(weekdaycols)

# tmpdf = pd.get_dummies(data['weekday'])
# tmpdf.columns = weekdaycols
# data[weekdaycols] = tmpdf

# tmpdf = pd.get_dummies(data_test['weekday'])
# tmpdf.columns = weekdaycols
# data_test[weekdaycols] = tmpdf

# data.drop("weekday", axis=1, inplace=True)
# data_test.drop("weekday", axis=1, inplace=True)

In [12]:
#data_test['day'] = data_test['date'].apply(lambda x: int(str(x)[6:8]))
data.fillna(0, inplace=True)
data_test.fillna(0, inplace=True)

In [13]:
# X = data.loc[:,'date':].as_matrix()
# y = data.loc[:,'label'].as_matrix()
# X_test = data_test.loc[:,'date':].as_matrix()
data_train = data[data['date'] <= 20171031]
data_val = data[data['date'] > 20171031]
# data_train = data[:int(data.shape[0]*0.8)]
# data_val = data[int(data.shape[0]*0.8):]
print(data.shape)
print(data_train.shape)
print(data_val.shape)

(994731, 300)
(911606, 300)
(83125, 300)


In [14]:
data_train

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,1,20170917,0,1,1,1,100805.0,1,5,2,1,1,0,2,1,2,2,1,1,2,0,14.0,13.0,48.0,48.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,4.0,4.0,5.0,1.0,1.0,4.0,3.0,3.0,79.0,12.0,302.0,32.0,301.0,302.0,302.0,302.0,23.0,26.0,23.0,26.0,2.0,2.0,3.0,4.0,23.0,26.0,5.0,717.4,733.3,231.8,472.8,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,4.0,8.0,11.0,12.0,4.0,12.0,23.0,26.0,27.0,5.0,12.0,24.0,27.0,122.0,121.0,16.0,121.0,121.0,122.0,121.0,122.0,121.0,16.0,121.0,122.0,32.0,31.0,13.0,27.0,26.0,27.0,32.0,79.0,40.0,39.0,39.0,63.0,31.0,31.0,15.0,28.0,29.0,29.0,31.0,231.0,232.0,17.0,99.0,99.0,98.0,232.0,121.0,122.0,17.0,121.0,121.0,302.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,5.0,12.0,24.0,26.0,27.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,13.0,158.0,301.0,335.0,398.0,4.0,54.0,109.0,112.0,113.0,302.0,324.0,391.0,13.0,302.0,160.0,160.0,161.0,8.0,160.0
2,1270cb8a85eedd57672b2c6297fa5633e36773a2c3a351...,1,20171022,0,0,1,0,100102.0,0,6,1,0,0,0,2,1,2,1,1,2,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,15.0,18.0,37.0,102.0,139.0,206.0,253.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,7.0,13.0,2.0,1.0,2.0,4.0,6.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9fa009724ee7ff9

In [15]:
data_val

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,1,20171103,0,0,0,0,100807.0,0,5,1,1,1,1,2,0,2,0,2,2,2,1,31.0,61.0,142.0,245.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,96.0,38.0,301.0,302.0,302.0,302.0,301.0,302.0,40.0,79.0,40.0,79.0,2.0,3.0,4.0,3.0,41.0,80.0,96.5,721.1,3252.0,154.5,394.5,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,8.0,15.0,30.0,4.0,14.0,34.0,70.0,142.0,3.0,15.0,41.0,79.0,191.0,192.0,190.0,190.0,190.0,190.0,191.0,192.0,191.0,190.0,190.0,190.0,32.0,33.0,25.0,25.0,24.0,25.0,31.0,79.0,42.0,42.0,42.0,68.0,32.0,32.0,28.0,27.0,28.0,27.0,32.0,235.0,235.0,126.0,126.0,127.0,126.0,235.0,192.0,191.0,190.0,189.0,189.0,301.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,14.0,41.0,80.0,167.0,2.0,2.0,4.0,3.0,3.0,4.0,6.0,6.0,2.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,4.0,1.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,10.0,16.0,2.0,2.0,4.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,27.0,297.0,302.0,384.0,770.0,20.0,99.0,102.0,124.0,140.0,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
10,741b5ae5f9f722faca496194ace8bc7920edb8fa1d4477...,1,20171104,0,0,1,1,100807.0,0,1,1,1,2,1,2,2,2,1,0,2,2,0,32.0,62.0,171.0,284.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,95.0,94.0,95.0,95.0,95.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,3.0,2.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,96.0,97.0,101.0,110.0,114.0,199.0,215.0,2.0,1.0,1.0,2.0,95.0,95.0,95.0,94.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,95.0,94.0,94.0,94.0,94.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,0.0,54.0,257.0,302.0,354.0,806.0,32.0,113.0,120.0,128.0,134.0,

In [16]:
data_val['label'].value_counts()

1    81804
0     1321
Name: label, dtype: int64

In [14]:
X = data.loc[:,'f1':].as_matrix()
y = data.loc[:,'label'].as_matrix()

X_train = data_train.loc[:,'f1':].as_matrix()
y_train = data_train.loc[:,'label'].as_matrix()

X_val = data_val.loc[:,'f1':].as_matrix()
y_val = data_val.loc[:,'label'].as_matrix()

X_test = data_test.loc[:,'f1':].as_matrix()

In [17]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
print(X.shape)
print(y.shape)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)

(994731, 297)
(994731,)
(911606, 297)
(911606,)
(83125, 297)
(83125,)
(500538, 297)


In [16]:
del data, data_train, data_val
gc.collect()

125

In [24]:
def eval_func(y_pred, y_true):
    fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
    score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
        fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
    return 'score', score, True

# def eval_func(y_true, y_pred):
#     y_pred = y_pred.get_label()
#     fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
#     score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
#         fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
#     return 'score', score, True

In [18]:
# lgb_train = lgb.Dataset(X_train, y_train)
# lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
# params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': {'l2', 'auc'},
#     'num_leaves': 50,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0
# }
# gbm = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=300,
#                 valid_sets=lgb_eval,
#                 early_stopping_rounds=5)

[1]	valid_0's l2: 0.226662	valid_0's auc: 0.948663
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.205662	valid_0's auc: 0.956029
[3]	valid_0's l2: 0.186734	valid_0's auc: 0.965446
[4]	valid_0's l2: 0.169644	valid_0's auc: 0.965758
[5]	valid_0's l2: 0.154231	valid_0's auc: 0.965925
[6]	valid_0's l2: 0.140326	valid_0's auc: 0.966829
[7]	valid_0's l2: 0.12776	valid_0's auc: 0.96801
[8]	valid_0's l2: 0.116422	valid_0's auc: 0.968056
[9]	valid_0's l2: 0.106191	valid_0's auc: 0.967999
[10]	valid_0's l2: 0.0969397	valid_0's auc: 0.968085
[11]	valid_0's l2: 0.0885938	valid_0's auc: 0.968262
[12]	valid_0's l2: 0.0810643	valid_0's auc: 0.968577
[13]	valid_0's l2: 0.0742681	valid_0's auc: 0.968538
[14]	valid_0's l2: 0.0681218	valid_0's auc: 0.968448
[15]	valid_0's l2: 0.0625789	valid_0's auc: 0.968556
[16]	valid_0's l2: 0.0575706	valid_0's auc: 0.968581
[17]	valid_0's l2: 0.0530224	valid_0's auc: 0.9685
[18]	valid_0's l2: 0.0489104	valid_0's auc: 0.968523
[19]	va

In [18]:
# clf = lgb.LGBMClassifier(   
#                             boosting_type='gbdt',
#                             num_leaves=50,
#                             n_estimators=300,
#                             objective='binary',
#                             learning_rate=0.05,
#                             feature_fraction=0.9,
#                             bagging_fraction=0.8,
#                             random_state=2018,
#                             bagging_freq=5,
#                             verbose=0,
#                             n_jobs=-1     
#                         )
# clf.fit(X_train, y_train, eval_set=[(X_val, y_val), (X_train, y_train)], eval_metric={'binary_logloss','auc'}, early_stopping_rounds=50)

[1]	valid_0's auc: 0.947423	valid_0's binary_logloss: 0.646435	valid_1's auc: 0.934721	valid_1's binary_logloss: 0.646633
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.948677	valid_0's binary_logloss: 0.604192	valid_1's auc: 0.937038	valid_1's binary_logloss: 0.604511
[3]	valid_0's auc: 0.956911	valid_0's binary_logloss: 0.565785	valid_1's auc: 0.946771	valid_1's binary_logloss: 0.566171
[4]	valid_0's auc: 0.960035	valid_0's binary_logloss: 0.530731	valid_1's auc: 0.948396	valid_1's binary_logloss: 0.531215
[5]	valid_0's auc: 0.960264	valid_0's binary_logloss: 0.498624	valid_1's auc: 0.949592	valid_1's binary_logloss: 0.499151
[6]	valid_0's auc: 0.962325	valid_0's binary_logloss: 0.469105	valid_1's auc: 0.953134	valid_1's binary_logloss: 0.469726
[7]	valid_0's auc: 0.962761	valid_0's binary_logloss: 0.441922	valid_1's auc: 0.953784	valid_1's binary_logloss: 0.44263
[8]	valid_0's auc: 0.963138	valid_0's binary_logloss: 0.416746	valid_1's auc: 0.95406

[67]	valid_0's auc: 0.975378	valid_0's binary_logloss: 0.0501716	valid_1's auc: 0.976819	valid_1's binary_logloss: 0.0505166
[68]	valid_0's auc: 0.975462	valid_0's binary_logloss: 0.0493779	valid_1's auc: 0.97695	valid_1's binary_logloss: 0.0496818
[69]	valid_0's auc: 0.975561	valid_0's binary_logloss: 0.0486244	valid_1's auc: 0.977102	valid_1's binary_logloss: 0.0489026
[70]	valid_0's auc: 0.975594	valid_0's binary_logloss: 0.0479243	valid_1's auc: 0.977202	valid_1's binary_logloss: 0.0481713
[71]	valid_0's auc: 0.975724	valid_0's binary_logloss: 0.0472534	valid_1's auc: 0.977478	valid_1's binary_logloss: 0.0474498
[72]	valid_0's auc: 0.975712	valid_0's binary_logloss: 0.0466295	valid_1's auc: 0.977595	valid_1's binary_logloss: 0.0467845
[73]	valid_0's auc: 0.975713	valid_0's binary_logloss: 0.0460431	valid_1's auc: 0.977663	valid_1's binary_logloss: 0.0461587
[74]	valid_0's auc: 0.975755	valid_0's binary_logloss: 0.0454529	valid_1's auc: 0.978198	valid_1's binary_logloss: 0.0455577
[

[133]	valid_0's auc: 0.97965	valid_0's binary_logloss: 0.034835	valid_1's auc: 0.985645	valid_1's binary_logloss: 0.0326298
[134]	valid_0's auc: 0.97966	valid_0's binary_logloss: 0.0348026	valid_1's auc: 0.985726	valid_1's binary_logloss: 0.0325537
[135]	valid_0's auc: 0.979695	valid_0's binary_logloss: 0.0347545	valid_1's auc: 0.985795	valid_1's binary_logloss: 0.0324654
[136]	valid_0's auc: 0.979769	valid_0's binary_logloss: 0.0347175	valid_1's auc: 0.98589	valid_1's binary_logloss: 0.0323928
[137]	valid_0's auc: 0.979811	valid_0's binary_logloss: 0.0346656	valid_1's auc: 0.985991	valid_1's binary_logloss: 0.0323094
[138]	valid_0's auc: 0.979883	valid_0's binary_logloss: 0.0346283	valid_1's auc: 0.986087	valid_1's binary_logloss: 0.0322268
[139]	valid_0's auc: 0.979576	valid_0's binary_logloss: 0.034692	valid_1's auc: 0.986191	valid_1's binary_logloss: 0.0321568
[140]	valid_0's auc: 0.979649	valid_0's binary_logloss: 0.034658	valid_1's auc: 0.986354	valid_1's binary_logloss: 0.032083

[199]	valid_0's auc: 0.980579	valid_0's binary_logloss: 0.0341523	valid_1's auc: 0.98981	valid_1's binary_logloss: 0.029194
[200]	valid_0's auc: 0.980581	valid_0's binary_logloss: 0.0341521	valid_1's auc: 0.989842	valid_1's binary_logloss: 0.0291522
[201]	valid_0's auc: 0.980605	valid_0's binary_logloss: 0.0341358	valid_1's auc: 0.989875	valid_1's binary_logloss: 0.0291136
[202]	valid_0's auc: 0.980613	valid_0's binary_logloss: 0.0341344	valid_1's auc: 0.989898	valid_1's binary_logloss: 0.0290827
[203]	valid_0's auc: 0.980601	valid_0's binary_logloss: 0.0341399	valid_1's auc: 0.98994	valid_1's binary_logloss: 0.0290432
[204]	valid_0's auc: 0.980588	valid_0's binary_logloss: 0.034165	valid_1's auc: 0.989976	valid_1's binary_logloss: 0.0290115
[205]	valid_0's auc: 0.980602	valid_0's binary_logloss: 0.0341521	valid_1's auc: 0.990004	valid_1's binary_logloss: 0.0289827
[206]	valid_0's auc: 0.980568	valid_0's binary_logloss: 0.0341748	valid_1's auc: 0.990033	valid_1's binary_logloss: 0.0289

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=50, objective='binary', random_state=2018,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1, verbose=0)

In [19]:
# y_pred = clf.predict_proba(X_test)[:,0]
# result = pd.DataFrame({'id':data_test['id'].as_matrix(), 'score':y_pred})
# result.to_csv("result8.csv", index=False)

In [24]:
# count = 0
# white_samples = []
# for i in range(len(y_pred)):
#     if y_pred[i] < 0.0009:
#         count += 1
#         white_samples.append(i)
#         #y_pred[i] = 0
# print(count)

132235


In [25]:
#white_samples

[0,
 7,
 10,
 21,
 22,
 24,
 25,
 39,
 44,
 45,
 48,
 50,
 56,
 59,
 60,
 65,
 69,
 70,
 72,
 73,
 82,
 84,
 86,
 89,
 95,
 98,
 99,
 102,
 105,
 110,
 112,
 116,
 117,
 119,
 124,
 126,
 127,
 130,
 139,
 148,
 150,
 153,
 168,
 176,
 177,
 179,
 182,
 193,
 195,
 196,
 197,
 202,
 203,
 206,
 210,
 220,
 228,
 236,
 244,
 246,
 248,
 249,
 255,
 270,
 271,
 274,
 277,
 279,
 281,
 282,
 289,
 290,
 293,
 299,
 301,
 305,
 306,
 308,
 311,
 316,
 317,
 320,
 326,
 327,
 328,
 334,
 335,
 340,
 341,
 343,
 346,
 348,
 351,
 352,
 362,
 364,
 365,
 366,
 375,
 377,
 379,
 389,
 391,
 396,
 399,
 404,
 405,
 409,
 411,
 414,
 417,
 420,
 433,
 454,
 457,
 458,
 459,
 461,
 465,
 466,
 468,
 469,
 473,
 479,
 484,
 487,
 490,
 494,
 496,
 500,
 502,
 503,
 507,
 508,
 513,
 516,
 518,
 520,
 521,
 539,
 540,
 541,
 544,
 548,
 551,
 553,
 559,
 562,
 563,
 571,
 574,
 582,
 584,
 585,
 591,
 593,
 605,
 608,
 610,
 611,
 624,
 625,
 631,
 632,
 639,
 642,
 644,
 645,
 648,
 649,
 650,
 65

In [39]:
# count = 0
# for i in range(len(y_pred)):
#     if y_pred[i] == 0:
#         count += 1
# print(count)

132235


In [22]:
clf = GradientBoostingClassifier(
                                    n_estimators=300,
                                    subsample=0.8,
                                    max_leaf_nodes=50,
                                    max_features=0.9,
                                    learning_rate=0.05,
                                    random_state=2018,
                                    verbose=1
                                )
clf.fit(X, y)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.1525           0.0195          137.44m
         2           0.1439           0.0078          137.94m
         3           0.1398           0.0052          138.45m
         4           0.1357           0.0035          136.54m
         5           0.1325           0.0031          135.17m
         6           0.1302           0.0029          134.49m
         7           0.1264           0.0025          133.47m
         8           0.1252           0.0023          132.87m
         9           0.1228           0.0018          132.07m
        10           0.1210           0.0013          131.40m
        20           0.1074           0.0010          131.32m
        30           0.0994           0.0007          128.57m
        40           0.0942           0.0004          124.63m
        50           0.0903           0.0002          120.41m
        60           0.0881           0.0002          117.20m
       

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=3,
              max_features=0.9, max_leaf_nodes=50,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=2018, subsample=0.8, verbose=1,
              warm_start=False)

In [32]:
y_val_pred = clf.predict_proba(X_val)[:,1]

In [23]:
y_pred = clf.predict_proba(X_test)[:,0]
result = pd.DataFrame({'id':data_test['id'].as_matrix(), 'score':y_pred})
result.to_csv("result28.csv", index=False)